In [1]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from load_data import initialize_data
from reading_datasets import read_task5
from labels_to_ids import task5_labels_to_ids
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def train(epoch, train_loader, model, optimizer, device, grad_step = 1, max_grad_norm = 10):
    tr_loss, tr_accuracy = 0, 0
    tr_precision, tr_recall = 0, 0
    tr_f1score = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    optimizer.zero_grad()
    
    for idx, batch in enumerate(train_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        if (idx + 1) % 20 == 0:
            print('FINSIHED BATCH:', idx, 'of', len(train_loader))

        #loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        output = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += output[0]

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
        
        # Compute Precision
        tmp_tr_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0 )[2]
        tr_precision += tmp_tr_precision
        
        # Compute Recall
        tmp_tr_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0 )[2]
        tr_recall += tmp_tr_recall
        
        # Compute f1score
        tmp_tr_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(),labels=[0,1,2], average=None, zero_division=0)[2]
        tr_f1score += tmp_tr_f1score
    

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_grad_norm
        )
        
        # backward pass
        output['loss'].backward()
        if (idx + 1) % grad_step == 0:
            optimizer.step()
            optimizer.zero_grad()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    tr_precision = tr_precision / nb_tr_steps
    tr_recall = tr_recall / nb_tr_steps
    tr_f1score= tr_f1score / nb_tr_steps
    #print(f"Training loss epoch: {epoch_loss}")
    #print(f"Training accuracy epoch: {tr_accuracy}")

    return model

In [3]:
def testing(model, testing_loader, labels_to_ids, device):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_precision, eval_recall = 0, 0
    eval_f1score = 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
     
    
    ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            #loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            output = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += output['loss'].item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            
            # Compute Precision
            tmp_eval_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0)[2]
            eval_precision += tmp_eval_precision
            
            # Compute Recall
            tmp_eval_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), labels=[0,1,2], average = None, zero_division=0)[2]
            eval_recall += tmp_eval_recall
            
            # Compute f1score
            tmp_eval_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(),labels=[0,1,2], average=None, zero_division=0)[2]
            eval_f1score += tmp_eval_f1score

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    eval_precision = eval_precision / nb_eval_steps
    eval_recall = eval_recall / nb_eval_steps
    eval_f1score = eval_f1score / nb_eval_steps
    #print(f"Validation Loss: {eval_loss}")
    #print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_accuracy, eval_precision, eval_recall, eval_f1score

In [4]:
def main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location):
    #Initialization training parameters
    max_len = 256
    batch_size = 32
    grad_step = 1
    learning_rate = 1e-05
    initialization_input = (max_len, batch_size)

    #Reading datasets and initializing data loaders
    dataset_location = '../2022.07.07_task5/'

    train_data = read_task5(dataset_location , split = 'train_oversample')
    dev_data = read_task5(dataset_location , split = 'dev')
    #test_data = read_task5(dataset_location , split = 'dev')#load test set
    labels_to_ids = task5_labels_to_ids
    #input_data = (train_data, dev_data, labels_to_ids)

    #Define tokenizer, model and optimizer
    device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time
    if model_load_flag:
        tokenizer = AutoTokenizer.from_pretrained(model_load_location)
        model = AutoModelForSequenceClassification.from_pretrained(model_load_location)
    else: 
        tokenizer =  AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_to_ids))
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.to(device)

    #Get dataloaders
    train_loader = initialize_data(tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)
    dev_loader = initialize_data(tokenizer, initialization_input, dev_data, labels_to_ids, shuffle = True)
    #test_loader = initialize_data(tokenizer, initialization_input, test_data, labels_to_ids, shuffle = True)#create test loader

    best_dev_acc = 0
    best_test_acc = 0
    best_dev_precision = 0
    best_test_precision = 0
    best_dev_recall = 0
    best_test_recall = 0
    best_dev_f1score = 0
    best_test_f1score = 0
    best_epoch = -1
    
    list_dev_acc = [] 
    list_test_acc = []  
    list_dev_precision = []  
    list_test_precision  = []  
    list_dev_recall = []  
    list_test_recall = []  
    list_dev_f1score = []  
    list_test_f1score = []
    
    for epoch in range(n_epochs):
        start = time.time()
        print(f"Training epoch: {epoch + 1}")

        #train model
        if not model_load_flag:
            model = train(epoch, train_loader, model, optimizer, device, grad_step)
        
        #testing and logging
        labels_dev, predictions_dev, dev_accuracy, dev_precision, dev_recall, dev_f1score = testing(model, dev_loader, labels_to_ids, device)
        print('DEV ACC:', dev_accuracy)
        print('DEV Precision:' , dev_precision)
        print('DEV Recall:' , dev_recall)
        print('DEV F1Score:' , dev_f1score)
        
        list_dev_acc.append(dev_accuracy)     
        list_dev_precision.append(dev_precision)   
        list_dev_recall.append(dev_recall)  
        list_dev_f1score.append(dev_f1score)  
        
        
        #labels_test, predictions_test, test_accuracy, test_precision, test_recall, test_f1score = testing(model, test_loader, labels_to_ids, device)
        #print('TEST ACC:', test_accuracy)
        #print('TEST Precision:' , test_precision)
        #print('TEST Recall:' , test_recall)
        #print('TEST F1Score:' , test_f1score)
        
        #list_test_acc.append(test_accuracy) 
        #list_test_precision.append(test_precision)  
        #list_test_recall.append(test_recall)
        #list_test_f1score.append(test_f1score) 

        #saving model
        if dev_accuracy > best_dev_acc:
            best_dev_acc = dev_accuracy
            #best_test_acc = test_accuracy
        if dev_precision > best_dev_precision:
            best_dev_precision = dev_precision
            #best_test_precision = test_precision
        if dev_recall > best_dev_recall:
            best_dev_recall = dev_recall
            #best_test_recall = test_recall
        if dev_f1score > best_dev_f1score:
            best_dev_f1score = dev_f1score
            #best_test_f1score = test_f1score
            best_epoch = epoch
            
            if model_save_flag:
                os.makedirs(model_save_location, exist_ok=True)
                tokenizer.save_pretrained(model_save_location)
                model.save_pretrained(model_save_location)

        now = time.time()
        print('BEST ACCURACY --> ', 'DEV:', round(best_dev_acc, 5))
        print('BEST PRECISION --> ', 'DEV:', round(best_dev_precision, 5))
        print('BEST RECALL --> ', 'DEV:', round(best_dev_recall, 5))
        print('BEST F1SCORE --> ', 'DEV:', round(best_dev_f1score, 5))
        print('TIME PER EPOCH:', (now-start)/60 )
        print()

    return best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score

In [5]:
if __name__ == '__main__':
    n_epochs = 5
    models = ['bert-base-multilingual-uncased']
    
    #model saving parameters
    model_save_flag = True
    model_load_flag = False
    
    overall_list_dev_acc = [] 
    overall_list_test_acc = []    
    overall_list_dev_precision = []  
    overall_list_test_precision  = []  
    overall_list_dev_recall = []  
    overall_list_test_recall = []  
    overall_list_dev_f1score = []  
    overall_list_test_f1score = [] 
    
    for i in range(5):
        
        for model_name in models:

            model_save_location = 'saved_models/' + model_name + '_oversampled' + str(i)
            model_load_location = None

            best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score = main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location)
            
            overall_list_dev_acc.append(list_dev_acc) 
            overall_list_test_acc.append(list_test_acc) 
            overall_list_dev_precision.append(list_dev_precision)  
            overall_list_test_precision.append(list_test_precision) 
            overall_list_dev_recall.append(list_dev_recall)  
            overall_list_test_recall.append(list_test_recall)  
            overall_list_dev_f1score.append(list_dev_f1score)  
            overall_list_test_f1score.append(list_test_f1score) 

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 449
FINSIHED BATCH: 39 of 449
FINSIHED BATCH: 59 of 449
FINSIHED BATCH: 79 of 449
FINSIHED BATCH: 99 of 449
FINSIHED BATCH: 119 of 449
FINSIHED BATCH: 139 of 449
FINSIHED BATCH: 159 of 449
FINSIHED BATCH: 179 of 449
FINSIHED BATCH: 199 of 449
FINSIHED BATCH: 219 of 449
FINSIHED BATCH: 239 of 449
FINSIHED BATCH: 259 of 449
FINSIHED BATCH: 279 of 449
FINSIHED BATCH: 299 of 449
FINSIHED BATCH: 319 of 449
FINSIHED BATCH: 339 of 449
FINSIHED BATCH: 359 of 449
FINSIHED BATCH: 379 of 449
FINSIHED BATCH: 399 of 449
FINSIHED BATCH: 419 of 449
FINSIHED BATCH: 439 of 449
Validation loss per 100 evaluation steps: 0.3942628502845764
DEV ACC: 0.7815990593768372
DEV Precision: 0.554152129642326
DEV Recall: 0.987925170068027
DEV F1Score: 0.6931911461435932
BEST ACCURACY -->  DEV: 0.7816
BEST PRECISION -->  DEV: 0.55415
BEST RECALL -->  DEV: 0.98793
BEST F1SCORE -->  DEV: 0.69319
TIME PER EPOCH: 11.61164412498474

Training epoch: 2
FINSIHED BATCH: 19 of 449
FINSI

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 449
FINSIHED BATCH: 39 of 449
FINSIHED BATCH: 59 of 449
FINSIHED BATCH: 79 of 449
FINSIHED BATCH: 99 of 449
FINSIHED BATCH: 119 of 449
FINSIHED BATCH: 139 of 449
FINSIHED BATCH: 159 of 449
FINSIHED BATCH: 179 of 449
FINSIHED BATCH: 199 of 449
FINSIHED BATCH: 219 of 449
FINSIHED BATCH: 239 of 449
FINSIHED BATCH: 259 of 449
FINSIHED BATCH: 279 of 449
FINSIHED BATCH: 299 of 449
FINSIHED BATCH: 319 of 449
FINSIHED BATCH: 339 of 449
FINSIHED BATCH: 359 of 449
FINSIHED BATCH: 379 of 449
FINSIHED BATCH: 399 of 449
FINSIHED BATCH: 419 of 449
FINSIHED BATCH: 439 of 449
Validation loss per 100 evaluation steps: 0.2978043258190155
DEV ACC: 0.8445583480305702
DEV Precision: 0.6874527588813302
DEV Recall: 0.8342970521541947
DEV F1Score: 0.734097210634141
BEST ACCURACY -->  DEV: 0.84456
BEST PRECISION -->  DEV: 0.68745
BEST RECALL -->  DEV: 0.8343
BEST F1SCORE -->  DEV: 0.7341
TIME PER EPOCH: 11.774381697177887

Training epoch: 2
FINSIHED BATCH: 19 of 449
FINS

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 449
FINSIHED BATCH: 39 of 449
FINSIHED BATCH: 59 of 449
FINSIHED BATCH: 79 of 449
FINSIHED BATCH: 99 of 449
FINSIHED BATCH: 119 of 449
FINSIHED BATCH: 139 of 449
FINSIHED BATCH: 159 of 449
FINSIHED BATCH: 179 of 449
FINSIHED BATCH: 199 of 449
FINSIHED BATCH: 219 of 449
FINSIHED BATCH: 239 of 449
FINSIHED BATCH: 259 of 449
FINSIHED BATCH: 279 of 449
FINSIHED BATCH: 299 of 449
FINSIHED BATCH: 319 of 449
FINSIHED BATCH: 339 of 449
FINSIHED BATCH: 359 of 449
FINSIHED BATCH: 379 of 449
FINSIHED BATCH: 399 of 449
FINSIHED BATCH: 419 of 449
FINSIHED BATCH: 439 of 449
Validation loss per 100 evaluation steps: 0.47305619716644287
DEV ACC: 0.7748750734861846
DEV Precision: 0.5766287944859374
DEV Recall: 0.9834656084656085
DEV F1Score: 0.7141800214526786
BEST ACCURACY -->  DEV: 0.77488
BEST PRECISION -->  DEV: 0.57663
BEST RECALL -->  DEV: 0.98347
BEST F1SCORE -->  DEV: 0.71418
TIME PER EPOCH: 11.763378171126048

Training epoch: 2
FINSIHED BATCH: 19 of 449


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 449
FINSIHED BATCH: 39 of 449
FINSIHED BATCH: 59 of 449
FINSIHED BATCH: 79 of 449
FINSIHED BATCH: 99 of 449
FINSIHED BATCH: 119 of 449
FINSIHED BATCH: 139 of 449
FINSIHED BATCH: 159 of 449
FINSIHED BATCH: 179 of 449
FINSIHED BATCH: 199 of 449
FINSIHED BATCH: 219 of 449
FINSIHED BATCH: 239 of 449
FINSIHED BATCH: 259 of 449
FINSIHED BATCH: 279 of 449
FINSIHED BATCH: 299 of 449
FINSIHED BATCH: 319 of 449
FINSIHED BATCH: 339 of 449
FINSIHED BATCH: 359 of 449
FINSIHED BATCH: 379 of 449
FINSIHED BATCH: 399 of 449
FINSIHED BATCH: 419 of 449
FINSIHED BATCH: 439 of 449
Validation loss per 100 evaluation steps: 0.24601106345653534
DEV ACC: 0.7962962962962963
DEV Precision: 0.6549333823143346
DEV Recall: 0.8988662131519274
DEV F1Score: 0.7414884978241897
BEST ACCURACY -->  DEV: 0.7963
BEST PRECISION -->  DEV: 0.65493
BEST RECALL -->  DEV: 0.89887
BEST F1SCORE -->  DEV: 0.74149
TIME PER EPOCH: 11.770898111661275

Training epoch: 2
FINSIHED BATCH: 19 of 449
F

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 449
FINSIHED BATCH: 39 of 449
FINSIHED BATCH: 59 of 449
FINSIHED BATCH: 79 of 449
FINSIHED BATCH: 99 of 449
FINSIHED BATCH: 119 of 449
FINSIHED BATCH: 139 of 449
FINSIHED BATCH: 159 of 449
FINSIHED BATCH: 179 of 449
FINSIHED BATCH: 199 of 449
FINSIHED BATCH: 219 of 449
FINSIHED BATCH: 239 of 449
FINSIHED BATCH: 259 of 449
FINSIHED BATCH: 279 of 449
FINSIHED BATCH: 299 of 449
FINSIHED BATCH: 319 of 449
FINSIHED BATCH: 339 of 449
FINSIHED BATCH: 359 of 449
FINSIHED BATCH: 379 of 449
FINSIHED BATCH: 399 of 449
FINSIHED BATCH: 419 of 449
FINSIHED BATCH: 439 of 449
Validation loss per 100 evaluation steps: 0.47651007771492004
DEV ACC: 0.8039204879482658
DEV Precision: 0.6890125747268605
DEV Recall: 0.8449540759064568
DEV F1Score: 0.7369498404634196
BEST ACCURACY -->  DEV: 0.80392
BEST PRECISION -->  DEV: 0.68901
BEST RECALL -->  DEV: 0.84495
BEST F1SCORE -->  DEV: 0.73695
TIME PER EPOCH: 11.741074844201405

Training epoch: 2
FINSIHED BATCH: 19 of 449


In [6]:
print(best_dev_acc)

0.8276014109347443


In [7]:
print(best_epoch)

0


In [8]:
print(best_dev_precision)

0.6890125747268605


In [9]:
print(best_dev_recall)

0.8724867724867726


In [10]:
print(best_dev_f1score)

0.7369498404634196


In [11]:
print(overall_list_dev_acc)

[[0.7815990593768372, 0.8335537918871252, 0.8213734567901235, 0.8175338036449147, 0.8175889182833628], [0.8445583480305702, 0.8217408877131099, 0.8160089653145209, 0.8199772192827749, 0.8156047912992358], [0.7748750734861846, 0.8311654908877131, 0.8143004115226337, 0.8149250440917107, 0.8060883303938859], [0.7962962962962963, 0.8260214579659024, 0.817717519106408, 0.8136206643151088, 0.8273258377425045], [0.8039204879482658, 0.8276014109347443, 0.7963881540270429, 0.7995480599647267, 0.7836750440917107]]


In [12]:
print(overall_list_dev_precision)

[[0.554152129642326, 0.6065972853787979, 0.6528484963008772, 0.6042305137543232, 0.6327034517510706], [0.6874527588813302, 0.6625547292213961, 0.6433421516754853, 0.620669277812135, 0.6407501669406432], [0.5766287944859374, 0.6562982432030051, 0.6852229780801208, 0.5888699924414212, 0.6472411186696903], [0.6549333823143346, 0.5765111872254728, 0.5897134611420325, 0.6407092466616277, 0.6534362903410524], [0.6890125747268605, 0.6003459062982873, 0.6295758033853273, 0.5505102040816325, 0.6565444696397078]]


In [13]:
print(overall_list_dev_recall)

[[0.987925170068027, 0.9331002771478961, 0.8449924414210128, 0.6821296868915917, 0.6569160997732424], [0.8342970521541947, 0.8449655282988616, 0.7548122952884859, 0.70194347099109, 0.8107268833459309], [0.9834656084656085, 0.8393727524679907, 0.6287100025195264, 0.775925925925926, 0.5685122197026958], [0.8988662131519274, 0.9347694633408918, 0.7323112073112074, 0.6882401108591585, 0.7719496552829886], [0.8449540759064568, 0.8724867724867726, 0.7126543209876545, 0.6393383952907762, 0.5110040312421265]]


In [14]:
print(overall_list_dev_f1score)

[[0.6931911461435932, 0.7164747107989162, 0.713604171749535, 0.6202135992173745, 0.6218114086240284], [0.734097210634141, 0.7282012978312548, 0.6706789242740696, 0.6314982265305643, 0.7008788950223418], [0.7141800214526786, 0.7179990120435863, 0.6325758793545907, 0.6537722170480535, 0.5664418473942284], [0.7414884978241897, 0.6994596294769679, 0.6344552806628713, 0.6385148516801179, 0.691123188462124], [0.7369498404634196, 0.6978424973030517, 0.6384247435257313, 0.5726272983054643, 0.5489769489769489]]


In [15]:
#Best model is model 3